In [10]:
import pandas as pd
import xlrd
import os

In [7]:
df = pd.read_excel(rf'C:\APIPred\APIPred-main\APIPred-main\data\INPUT-APTAMER-PROTEIN-PAIRS.xls', header=1).drop(['ID', 'Class label'], axis=1)
df

,DNA/RNA sequence,Target_ID,Protein sequence
0,UCGGAGGUGGUUCAGCUCUGCAUCGACAGUUGGC,dsRNA_activated_protein_kinase,MDSTNYISKLFEYAQRQGQISDIKFEEVGTDGPDHLKTFTLRVVIK...
1,CGGGGTGTTGTCCTGTGCTCTCCGGAGAGCACAGGACAACACCCCG,Human_Nuclear_Factor_kB_RelA_p65,MDELFPLIFPAEPAQASGPYVEIIEQPKQRGMRFRYKCEGRSAGSI...
2,GCTGCAGTTGCACTGAATTCGCCTCTCGCCTCCGTACACTTAGTCG...,atSPL14,MDEVGAQVAAPMFIHQSLGRKRDLYYPMSNRLVQSQPQRRDEWNSK...
3,TATTTGCCCTTGCAGGCCGCAGGAGTGCTAGCAGT,HIV-1-RT,PISPIDTVPVKLKPGMDGPKVKQWPLTEEKIKALTEICNEMEKEGK...
4,UCUCUGGGCUCUUAGGAGAACGGAUAGGAGUGUGCUCGCU,prothrombin,LQENFLPQPRQQHHGTLVLHYRPHREEAGMQHPCLWPGSSHCSDDS...
...,...,...,...
2895,CCUGCCCUUUCCAACACCCGCCGAAACUCA,AlkB,MLDLFADAEPWQEPLAAGAVILRRFAFNAAEQLIRDINDVASQSPF...
2896,GCUUGGUACCGAGCUCGGAUCCACGUAGUAACGGGCCGCCAGUGUG...,Beta-catenin,MATQADLMELDMAMEPDRKAAVSHWQQQSYLDSGIHSGATTTAPSL...
2897,CTTCTGCCCGCCTCCTTCCTCGGGGTTAGAACGTCGTGTAGGACTC...,E.Coli_rho_factor,MNLTELKNTPVSELITLGENMGLENLARMRKQDIIFAILKQHAKSG...
2898,ACGUGCUCUCAUCUUUUAAUUCGUGGGCUCUGCGGCUAGCCUCUUA...,HIV-1_Rev_binding_protein,MAGRSGDSDEELLKAVRLIKFLYQSNPPPNPEGTRQARRNRRRRWR...


In [ ]:
#поменять пути к файлам
!python get_descriptors.py

Ошибка для последовательности PISPIDTVPVKLKPGMDGPKVKQWPLTEEKIKALTEICNEMEKEGKISKIGPENPYNTPVFAIKKKDSNKWRKLVDFRELNKRTQDFWEVQLGIPHPAGLKKKKSVTVLDVGDAYFSVPLDESFRKYTAFTIPSINNETPGVRYQYNVLPQGWKGSPAIFQCSMTKILEPXRXKNPEMVIYQYMDDLYVGSDLEIGQHRAKIEELRSHLLSWGFTTPDKKHQKEPPFLWMGYELHPDSWTVQPIELPEKDSWTVNE: Последовательность содержит неизвестные аминокислоты
Ошибка для последовательности XQKLMPNFSDSFGGGSGAGAGGGGMFGSGGGGGSTGSPGPGYGYSNYGFPPYGGITFHPGVTKSNAGVTHGTINTKFKNGPKDCAKSDDEESLTLPEKETEGEGPSLPMACTKTEPIALASTMEDKEQDMGFQDNLFLEKALQLARRHANALFDYAVTGDVKMLLAVQRHLTAVQDENGDSVLHLAIIHLHAQLVRDLLEVTSGLISDDIINMRNDLYQTPLHLAVITKQEDVVEDLLRVGADLSLLDRWGNSVLHLAAKEGHDRILSILLKSRKAAPLIDHPNGEGLNAIHIAVMSNSLPCLLLLVAAGAEVNAQEQKSGRTALHLAVEYDNISLAGCLLLEGDAHVDSTTYDGTTPLHIAAGRGSTRLAALLKAAGADPLVENFEPLYDLDDSWEKAGEDEGVVPGTTPLDMAANWQVFDILNGKPYEPVFTSDDILPQGDMKQLTEDTRLQLCKLLEIPDPDKNWATLAQKLGLGILNNAFRLSPAPSKTLMDNYEMNSGIVTASVTVVWRHPSANSALQSLLLETAHCYL: Последовательность содержит неизвестные аминокислоты
Ошибка для последовательности PISPIDTVPV

In [ ]:
import pandas as pd
import joblib
import os
from sklearn.feature_selection import SelectFromModel


script_dir = os.getcwd()

try:
    clf_fs = joblib.load(os.path.join(script_dir, 'models', 'feature_selector_model.pkl'))
    poly = joblib.load(os.path.join(script_dir, 'models', 'polynomial_transformer.pkl'))
    best_model = joblib.load(os.path.join(script_dir, 'models', 'xgboost_best_model.pkl'))
    config = joblib.load(os.path.join(script_dir, 'models', 'preprocessing_config.pkl'))
    features_list = joblib.load(os.path.join(script_dir, 'models', 'features_list.pkl'))
except FileNotFoundError as e:
    print(f"Ошибка: файл не найден. Проверьте путь: {e}")
except Exception as e:
    print(f"Ошибка при загрузке модели: {e}")


selection = SelectFromModel(
    estimator=clf_fs,
    threshold=config['threshold'], 
    prefit=config['prefit']
)


def preprocess_new_data(X_new):
 
    if set(X_new.columns) != set(features_list):
        print("Предупреждение: новые данные имеют другие признаки. Применяем преобразование.")
        X_new = X_new.reindex(columns=features_list, fill_value=0)
    

    X_selected = selection.transform(X_new)
    

    X_poly = poly.transform(X_selected)
    
    return X_poly

try:
    new_data = os.path.join(script_dir, 'data', 'dataset_with_descriptors.csv')
    new_X = pd.read_csv(new_data)
    #new_X = pd.read_csv(new_data).drop('Class', axis=1)
except FileNotFoundError as e:
    print(f"Ошибка: файл не найден. Проверьте путь: {e}")
except KeyError as e:
    print(f"Ошибка: столбец 'Class' отсутствует в данных: {e}")



try:
    X_new_preprocessed = preprocess_new_data(new_X)
    y_pred = best_model.predict(X_new_preprocessed)
    print("Предсказания:", y_pred)
    
    predictions_df = pd.DataFrame({
        'Predicted_Label': y_pred 
    })
    
    try:
        y_true = pd.read_csv(new_data)['Class']
        predictions_df['True_Label'] = y_true
    except KeyError:
        print("Столбец 'Class' отсутствует в данных. Истинные метки не добавлены.")
    
    output_path = os.path.join(script_dir, 'data', 'predictions.csv')
    predictions_df.to_csv(output_path, index=False)
    print(f"Предсказания сохранены в файл: {output_path}")
    
except Exception as e:
    print(f"Ошибка при предсказании: {e}")

Предупреждение: новые данные имеют другие признаки. Применяем преобразование.
Предсказания: [1 1 1 ... 0 0 0]
Столбец 'Class' отсутствует в данных. Истинные метки не добавлены.
Предсказания сохранены в файл: c:\APIPred\APIPred-main\APIPred-main\data\predictions.csv


c:\APIPred\.venv\lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
